In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,304 kB]
Ign:13 https://developer.download.nvidia.com/compute/

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-24 20:55:31--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-07-24 20:55:31 (6.53 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://seridane-bucket.s3.amazonaws.com/amazon_reviews_us_Grocery_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [18]:
# Create the vine_table dataFrame
total_vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body"])
vote_count = total_vine_df.filter("total_votes>20")
vote_count.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|          DO NOT BUY|These dates were ...|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|            Love it!|I love it.  Taste...|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|This is horrible ...|Thought I'd save ...|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|Floating "Dirt" o...|I was drinking a ...|
|R2XASGYIENUHQO|          5|           2

In [19]:
# Create a table where the data is filtered where the percentage of helpful_votes is equal to or greater than 50% 
vine_helpful50_df = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
vine_helpful50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|          DO NOT BUY|These dates were ...|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|            Love it!|I love it.  Taste...|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|This is horrible ...|Thought I'd save ...|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|Floating "Dirt" o...|I was drinking a ...|
|R2XASGYIENUHQO|          5|           2

In [20]:
# Create a df where the data is filtered to create a DataFrame or table where there is a Vine review
vine_review_df = vine_helpful50_df.filter(vine_helpful50_df["vine"] == "Y")
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|My Top Coconut Oi...|I'm very strict w...|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|I Love the Taste ...|I've long been a ...|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|      Almost perfect|I've been searchi...|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|Depends on what y...|Thai Kitchen Orga...|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|        Pretty tasty|I like these chil...|
| RX88JLUIFA4Q2|          4|           8

In [21]:
# Create a df where the data is filtered to create a DataFrame or table where there is NOT a Vine review
no_vine_review_df = vine_helpful50_df.filter(vine_helpful50_df["vine"] == "N")
no_vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|          DO NOT BUY|These dates were ...|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|            Love it!|I love it.  Taste...|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|This is horrible ...|Thought I'd save ...|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|Floating "Dirt" o...|I was drinking a ...|
|R2XASGYIENUHQO|          5|           2

In [36]:
# Total Number of Reviews
vine_helpful50_df.count()

26437

In [30]:
# Total 5-star reviews
five_star_count = vine_helpful50_df.filter(vine_helpful50_df["star_rating"]== 5)
five_star_count.count()

14648

In [34]:
# Percentage 5-star paid reviews
vine_fives = vine_review_df.filter(vine_review_df["star_rating"]== 5) 
vine_fives.count() / five_star_count.count()

0.0012288367012561442

In [35]:
# Percentage 5-star unpaid reviews
no_vine_fives = no_vine_review_df.filter(no_vine_review_df["star_rating"]== 5) 
no_vine_fives.count() / five_star_count.count()

0.9987711632987438

In [37]:
vine_fives.count()

18

In [38]:
no_vine_fives.count()

14630